### 

### 特征标准化

Standardization (标准化) / Z-score normization (零均值归一化，Z值归一化)

将特征归一化到**均值为 0，标准差为 1**的标准正态分布。

$$
\mu=0\ \ and\ \  \sigma=1
$$

$\mu$ 为均值，$\sigma$ 为标准差；样本标准值 (也被称为 $z$ 值) 计算如下：

$$
z = \frac{x - \mu}{\sigma}
$$

这一标准化过程，不仅在我们比较具有不同单位的测量值时很重要，同时也是许多机器学习算法的普遍要求。

**以梯度下降为例**。在逻辑回归，支持向量机，感知机，神经网络等模型中，通常会用到梯度下降这一优化算法。当特征在不同的规模时，部分特征的权重可能会比其他特征更新得更快，因为特征值 $x_j$ 也在权重更新时起到了作用:

$$
\Delta w_j = -\eta \frac{\partial J}{\partial w_j} = -\eta \sum_i (t^{(i)} - o^{(i)})x_{j}^{(i)}
$$

更新后的权重 $w_j := w_j + \Delta w_j$，其中 $\eta$ 是学习率, $t$ 是目标类标签，$o$ 是模型输出。

由于学习率 $\eta$ 固定，而每个特征的步长 $\Delta w_j$ 不一样，这就给选择合适的学习率带来了困难。当学习率过大时，步长小的特征权重会越过最优解，从而永不收敛；当学习率过大时，步长大的特征权重将需要更多步到达最优解，收敛速度放慢。

而当我们对各特征进行标准化后，它们的步长被缩放到了相同的规模，这个时候，选择一个合适的学习率，便能大大降低更新迭代次数。

![](images/feature_scaling1.png)

一个更加严谨的说法：

> Essentially, scaling the inputs (through mean normalization, or z-score) gives the error surface a more spherical shape, where it would otherwise be a very high curvature ellipse. Since gradient descent is curvature-ignorant, having an error surface with high curvature will mean that we take many steps which aren't necessarily in the optimal direction. When we scale the inputs, we reduce the curvature, which makes methods that ignore curvature (like gradient descent) work much better. When the error surface is circular (spherical), the gradient points right at the minimum, so learning is easy.
> 
> https://www.quora.com/Why-does-mean-normalization-help-in-gradient-descent

注:

- 梯度下降: gradient descent
- 逻辑回归: logistic regression
- 支持向量机: SVM (support vector machine)
- 感知机: perceptron
- 神经网络: neural network

**另一个 KNN 的例子**。KNN 模型使用欧氏距离计算点与点之间的距离，这一距离有严重收到各特征的计量单位的影响。例如假设我们有两个特征：身高 $x_1$，在 1.0~2.2 m 之间；体重 $x_2$，在 30~120 kg 之间。后者的量级接近前者的 20 倍。欧氏距离计算如下：

$$
distance(i, j) = \sqrt{(x_1^{(i)} - x_1^{(j)})^2 + (x_2^{(i)} - x_2^{(j)})^2}
$$

体重 $x_2$ 这一特征带来的差异远比身高 $x_1$ 要大，然而从常识来看，体重相差 1kg 的时候，身高相差会很小 (远小于 1m)。模型在聚类的时候，会更加关注体重。

用以下两张图作为解释。第一张图是未标准化的情况，样本分布在一条直线上，这很容易导致样本被错误分类，如图中所选点即被错误分类为红色。

![](images/feature_scaling2.png)

第二张图是标准化后的情况，样本均匀分布在二维空间中，我们可以更加清晰地看到两类的明确界限，图中所选点即被正确分类为蓝色。
![](images/feature_scaling3.png)

> https://stats.stackexchange.com/questions/287425/why-do-you-need-to-scale-data-in-knn

**特征标准化在以下模型中起到了关键作用**

- 利用欧式距离度量的 KNN 模型，如果你需要让所有的特征具有相同的贡献度
- K-means 聚类模型，同 KNN
- 逻辑回归，支持向量机，感知机，神经网络等，如果你使用基于梯度下降/上升的优化算法，否则一些权重的更新可能要比另一些要快
- 线性判别分析，主成分分析，核主成分分析，如果你需要找到最大化方差的方向（在那些方向/特征向量/主成分是正交的约束下）；你希望拥有相同取值范围的特征，因为算法会更加强调那些取值范围更大的特征的重要性。

**什么模型不需要特征标准化**

决策树相关的模型，特征标准化起不到什么作用。例如，考虑决策 $x_i >= c_1$ 和 $x_2 <= c_2$，$c_1$ 和 $c_2$ 为常量，由于 $x_1$ 和 $x_2$ 均只在自身的尺度范围内作比较，两者互不影响，这个时候，是否需要缩放就显得真不那么重要了。

### 最小-最大缩放

Min-Max Scaling，Z-score normalization 之外的另一种特征归一化方法，它将数据缩放到 $[0, 1]$ 范围内。

与标准化相比，这个有边界的方法具有更小的标准差，这可以抑制离群值的影响。最小最大缩放公式如下：

$$
X_{norm} = \frac{X-X_{min}}{X_{max}-X_{min}}
$$

**何时使用 Z-score normalization？何时使用 Min-Max Scaling？**

这并没有确切的答案，主要是具体应用而定。

例如，在聚类分析中，为了比较基于某些距离度量的要素之间的相似性，standardization 可能尤其重要。另一个突出的例子是主成分分析，在这种情况下，我们通常更喜欢标准化而不是最小-最大缩放，因为我们对最大化方差的成分感兴趣（取决于实际问题，以及 PCA 是否通过相关矩阵而不是协方差矩阵计算成分）。

然而，这并不代表 Min-Max Scaling 没有作用！例如，在图像处理中，需要将像素强度归一化到特定范围（比如 RGB 颜色范围 0~255)。此外，某些神经网络算法要求输入数据范围为 0-1。

In [1]:
import pandas as pd